# uCov case study

## Import and functions

In [1]:
import pandas as pd

In [2]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [3]:
# 
def unique_uses(ref_suf, comp_suf, cols=["usageKind","usageType","usageFullyQualifiedName"]):
    unique_suf = ref_suf[cols].drop_duplicates().merge(comp_suf[cols].drop_duplicates(), how='left', indicator=True)
    return unique_suf[unique_suf['_merge'] == 'left_only'][cols]

## Loading data

In [4]:
sum = pd.read_csv("JSoup-1.16.1.SUM.csv", sep="|")

# bug fix for initial ucov versions not adding "Method > VirtualInvocation" when a "Method > Invocation" is found
missing_data = sum[sum["usageKind"] == "SYMBOL_KIND_METHOD"][sum["usageType"] == "SYMBOL_USE_INVOCATION"]
missing_data['usageType'].replace({"SYMBOL_USE_INVOCATION": "SYMBOL_USE_VIRTUAL_INVOCATION"}, inplace=True)
sum = pd.concat([sum, missing_data], ignore_index=True)
sum.drop_duplicates(inplace=True)

sum['qualifiedUse'] = sum['usageKind'] + ' > ' + sum['usageType']
sum.head()

/tmp/ipykernel_5150/1918050568.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  missing_data = sum[sum["usageKind"] == "SYMBOL_KIND_METHOD"][sum["usageType"] == "SYMBOL_USE_INVOCATION"]


,projectId,projectType,projectLocation,usagePosition,usageEndPosition,usageFullyQualifiedName,usageKind,usageType,qualifiedUse
0,JSoup-1.16.1,MAIN,I:/UCov/Repro/jsoup/Main,/src/main/java/org/jsoup/select/elements.java(136:21),/src/main/java/org/jsoup/select/elements.java(141:5),org.jsoup.select.Elements.addClass(java.lang.String),SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,SYMBOL_KIND_METHOD > SYMBOL_USE_INVOCATION
1,JSoup-1.16.1,MAIN,I:/UCov/Repro/jsoup/Main,/src/main/java/org/jsoup/parser/tokenqueue.java(123:17),/src/main/java/org/jsoup/parser/tokenqueue.java(125:5),org.jsoup.parser.TokenQueue.consume(),SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,SYMBOL_KIND_METHOD > SYMBOL_USE_OVERRIDING
2,JSoup-1.16.1,MAIN,I:/UCov/Repro/jsoup/Main,/src/main/java/org/jsoup/select/evaluator.java(452:21),/src/main/java/org/jsoup/select/evaluator.java(460:6),"org.jsoup.select.Evaluator$CssNthEvaluator.matches(org.jsoup.nodes.Element,org.jsoup.nodes.Element)",SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,SYMBOL_KIND_METHOD > SYMBOL_USE_OVERRIDING
3,JSoup-1.16.1,MAIN,I:/UCov/Repro/jsoup/Main,/src/main/java/org/jsoup/nodes/entities.java(35:9),/src/main/java/org/jsoup/nodes/entities.java(35:40),org.jsoup.nodes.Entities$EscapeMode.xhtml,SYMBOL_KIND_FIELD,SYMBOL_USE_INSTANCE_FIELD_READ,SYMBOL_KIND_FIELD > SYMBOL_USE_INSTANCE_FIELD_READ
4,JSoup-1.16.1,MAIN,I:/UCov/Repro/jsoup/Main,/src/main/java/org/jsoup/nodes/node.java(808:16),/src/main/java/org/jsoup/nodes/node.java(811:5),org.jsoup.nodes.Node.hashCode(),SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,SYMBOL_KIND_METHOD > SYMBOL_USE_VIRTUAL_INVOCATION


In [5]:
suf = pd.read_csv("JSoup-1.16.1.SUF.csv", sep="|")

# bug fix for initial ucov versions not adding "Method > VirtualInvocation" when a "Method > Invocation" is found
missing_data = suf[suf["usageKind"] == "SYMBOL_KIND_METHOD"][suf["usageType"] == "SYMBOL_USE_INVOCATION"]
missing_data['usageType'].replace({"SYMBOL_USE_INVOCATION": "SYMBOL_USE_VIRTUAL_INVOCATION"}, inplace=True)
suf = pd.concat([suf, missing_data], ignore_index=True)
suf.drop_duplicates(inplace=True)

suf['qualifiedUse'] = suf['usageKind'] + ' > ' + suf['usageType']
suf['qualifiedSymbol'] = suf['usageFullyQualifiedName'] + ' > ' + suf['usageType']
suf.head()

/tmp/ipykernel_5150/3506722008.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  missing_data = suf[suf["usageKind"] == "SYMBOL_KIND_METHOD"][suf["usageType"] == "SYMBOL_USE_INVOCATION"]


,projectId,projectType,projectLocation,usagePosition,usageEndPosition,usageFullyQualifiedName,usageKind,usageType,qualifiedUse,qualifiedSymbol
0,JSoup-1.16.1,TEST,I:/UCov/Repro/jsoup/Tests,/src/test/java/org/jsoup/helper/datautiltest.java(105:32),/src/test/java/org/jsoup/helper/datautiltest.java(105:45),org.jsoup.nodes.Node.toString(),SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,SYMBOL_KIND_METHOD > SYMBOL_USE_INVOCATION,org.jsoup.nodes.Node.toString() > SYMBOL_USE_INVOCATION
1,JSoup-1.16.1,TEST,I:/UCov/Repro/jsoup/Tests,/src/test/java/org/jsoup/helper/datautiltest.java(115:102),/src/test/java/org/jsoup/helper/datautiltest.java(115:120),org.jsoup.parser.Parser.htmlParser(),SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,SYMBOL_KIND_METHOD > SYMBOL_USE_STATIC_INVOCATION,org.jsoup.parser.Parser.htmlParser() > SYMBOL_USE_STATIC_INVOCATION
2,JSoup-1.16.1,TEST,I:/UCov/Repro/jsoup/Tests,/src/test/java/org/jsoup/helper/datautiltest.java(115:9),/src/test/java/org/jsoup/helper/datautiltest.java(115:16),org.jsoup.nodes.Document,SYMBOL_KIND_TYPE,SYMBOL_USE_TYPE_REFERENCE,SYMBOL_KIND_TYPE > SYMBOL_USE_TYPE_REFERENCE,org.jsoup.nodes.Document > SYMBOL_USE_TYPE_REFERENCE
3,JSoup-1.16.1,TEST,I:/UCov/Repro/jsoup/Tests,/src/test/java/org/jsoup/helper/datautiltest.java(117:29),/src/test/java/org/jsoup/helper/datautiltest.java(117:38),org.jsoup.nodes.Document.body(),SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,SYMBOL_KIND_METHOD > SYMBOL_USE_INVOCATION,org.jsoup.nodes.Document.body() > SYMBOL_USE_INVOCATION
4,JSoup-1.16.1,TEST,I:/UCov/Repro/jsoup/Tests,/src/test/java/org/jsoup/helper/datautiltest.java(117:29),/src/test/java/org/jsoup/helper/datautiltest.java(117:45),org.jsoup.nodes.Element.text(),SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,SYMBOL_KIND_METHOD > SYMBOL_USE_INVOCATION,org.jsoup.nodes.Element.text() > SYMBOL_USE_INVOCATION


In [6]:
suf_test = suf[suf["projectType"] == "TEST"]
suf_sample = suf[suf["projectType"] == "SAMPLE"]
suf_client = suf[suf["projectType"] == "CLIENT"]

## Number of unique API interactions & unique API symbols

In [7]:
print(len(sum))
print(len(sum["usageFullyQualifiedName"].unique()))

2941
1138


## Number of API symbols & interactions used in SUFs

In [8]:
suf_symbols = suf["usageFullyQualifiedName"].unique()
suf_uses_unique = suf["qualifiedSymbol"].unique()
suf_client_symbols = suf_client["usageFullyQualifiedName"].unique()
suf_client_uses_unique = suf_client["qualifiedSymbol"].unique()
suf_test_symbols = suf_test["usageFullyQualifiedName"].unique()
suf_test_uses_unique = suf_test["qualifiedSymbol"].unique()
suf_sample_symbols = suf_sample["usageFullyQualifiedName"].unique()
suf_sample_uses_unique = suf_sample["qualifiedSymbol"].unique()
print(f'All: {len(suf_symbols)} used, {len(suf)} uses, {len(suf_uses_unique)} unique.')
print(f'Clients: {len(suf_client_symbols)} used, {len(suf_client)} uses, {len(suf_client_uses_unique)} unique.')
print(f'Tests: {len(suf_test_symbols)} used, {len(suf_test)} uses, {len(suf_test_uses_unique)} unique.')
print(f'Samples: {len(suf_sample_symbols)} used, {len(suf_sample)} uses, {len(suf_sample_uses_unique)} unique.')

All: 608 used, 29979 uses, 1028 unique.
Clients: 282 used, 15246 uses, 456 unique.
Tests: 586 used, 14511 uses, 967 unique.
Samples: 47 used, 222 uses, 69 unique.


## SUM Analysis

### Prevalent API interactions in the SUM (~= profile)

In [9]:
display(sum["qualifiedUse"].value_counts(normalize=True).to_frame().style.bar())

,qualifiedUse
SYMBOL_KIND_METHOD > SYMBOL_USE_VIRTUAL_INVOCATION,0.301598
SYMBOL_KIND_METHOD > SYMBOL_USE_OVERRIDING,0.300238
SYMBOL_KIND_METHOD > SYMBOL_USE_INVOCATION,0.223053
SYMBOL_KIND_METHOD > SYMBOL_USE_STATIC_INVOCATION,0.040462
SYMBOL_KIND_TYPE > SYMBOL_USE_TYPE_REFERENCE,0.038762
SYMBOL_KIND_CONSTRUCTOR > SYMBOL_USE_INVOCATION,0.033322
SYMBOL_KIND_CLASS > SYMBOL_USE_INSTANTIATION,0.025502
SYMBOL_KIND_CLASS > SYMBOL_USE_INHERITANCE,0.018021
SYMBOL_KIND_FIELD > SYMBOL_USE_INSTANCE_FIELD_READ,0.013261
SYMBOL_KIND_INTERFACE > SYMBOL_USE_IMPLEMENTATION,0.002720


### Unused API symbols

In [10]:
sum_symbols_not_seen = unique_uses(sum, suf, cols=["usageFullyQualifiedName"])
sum_symbols_not_seen_client = unique_uses(sum, suf_client, cols=["usageFullyQualifiedName"])
sum_symbols_not_seen_sample = unique_uses(sum, suf_sample, cols=["usageFullyQualifiedName"])
sum_symbols_not_seen_test = unique_uses(sum, suf_test, cols=["usageFullyQualifiedName"])
sum_uses_not_seen = unique_uses(sum, suf, cols=["qualifiedUse"])

print(f'There are {len(sum_uses_not_seen)} kinds of uses in the SUM unseen in the SUFs.');
display(sum_uses_not_seen.head(n=15))

print("Kinds of uses that are the least covered")
display(unique_uses(sum, suf, cols=["usageFullyQualifiedName", "usageType"]).groupby(["usageType"]).size().sort_values(ascending=False).to_frame().style.bar())

print(f'There are {len(sum_symbols_not_seen)} unused API symbols for all SUFs.');
display(sum_symbols_not_seen.head(n=15))
print(f'There are {len(sum_symbols_not_seen_client)} unused API symbols in clients.');
display(sum_symbols_not_seen_client.head(n=15))
print(f'There are {len(sum_symbols_not_seen_sample)} unused API symbols in samples.');
display(sum_symbols_not_seen_sample.head(n=15))
print(f'There are {len(sum_symbols_not_seen_test)} unused API symbols in tests.');
display(sum_symbols_not_seen_test.head(n=15))

There are 3 kinds of uses in the SUM unseen in the SUFs.


,qualifiedUse
2,SYMBOL_KIND_FIELD > SYMBOL_USE_INSTANCE_FIELD_READ
9,SYMBOL_KIND_FIELD > SYMBOL_USE_INSTANCE_FIELD_WRITE
11,SYMBOL_KIND_INTERFACE > SYMBOL_USE_EXTENSION


Kinds of uses that are the least covered


,0
usageType,
SYMBOL_USE_OVERRIDING,848
SYMBOL_USE_VIRTUAL_INVOCATION,442
SYMBOL_USE_INVOCATION,359
SYMBOL_USE_TYPE_REFERENCE,62
SYMBOL_USE_INHERITANCE,52
SYMBOL_USE_INSTANTIATION,50
SYMBOL_USE_STATIC_INVOCATION,48
SYMBOL_USE_INSTANCE_FIELD_READ,39
SYMBOL_USE_EXTENSION,8


There are 530 unused API symbols for all SUFs.


...

,usageFullyQualifiedName
0,org.jsoup.select.Elements.addClass(java.lang.String)
1,org.jsoup.parser.TokenQueue.consume()
2,"org.jsoup.select.Evaluator$CssNthEvaluator.matches(org.jsoup.nodes.Element,org.jsoup.nodes.Element)"
3,org.jsoup.nodes.Entities$EscapeMode.xhtml
4,org.jsoup.nodes.Node.hashCode()
6,org.jsoup.Connection$Request.timeout(int)
7,org.jsoup.nodes.Comment.nodeName()
8,org.jsoup.helper.ChangeNotifyingArrayList.removeAll(java.util.Collection)
11,org.jsoup.helper.HttpConnection.data(java.util.Collection)
12,org.jsoup.nodes.Document.charset(java.nio.charset.Charset)


There are 1091 unused API symbols in samples.


,usageFullyQualifiedName
1,org.jsoup.parser.TokenQueue.consume()
2,"org.jsoup.select.Evaluator$CssNthEvaluator.matches(org.jsoup.nodes.Element,org.jsoup.nodes.Element)"
3,org.jsoup.nodes.Entities$EscapeMode.xhtml
4,org.jsoup.nodes.Node.hashCode()
5,org.jsoup.select.Elements.outerHtml()
6,org.jsoup.Connection$Request.timeout(int)
7,org.jsoup.nodes.Comment.nodeName()
8,org.jsoup.helper.ChangeNotifyingArrayList.removeAll(java.util.Collection)
9,org.jsoup.Connection.referrer(java.lang.String)
10,org.jsoup.nodes.Node.traverse(org.jsoup.select.NodeVisitor)


There are 552 unused API symbols in tests.


,usageFullyQualifiedName
1,org.jsoup.parser.TokenQueue.consume()
2,"org.jsoup.select.Evaluator$CssNthEvaluator.matches(org.jsoup.nodes.Element,org.jsoup.nodes.Element)"
3,org.jsoup.nodes.Entities$EscapeMode.xhtml
6,org.jsoup.Connection$Request.timeout(int)
8,org.jsoup.helper.ChangeNotifyingArrayList.removeAll(java.util.Collection)
11,org.jsoup.helper.HttpConnection.data(java.util.Collection)
14,org.jsoup.helper.HttpConnection$Request.postDataCharset()
17,org.jsoup.select.Evaluator$IsEmpty
18,org.jsoup.nodes.Attribute.clone()
20,"org.jsoup.nodes.Range$Position(int,int,int)"


### Unused API interactions in all SUFs

In [11]:
sum_not_seen = unique_uses(sum, suf)
print(len(sum_not_seen))
display(sum_not_seen.head(n=15))

1913


,usageKind,usageType,usageFullyQualifiedName
1,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,org.jsoup.parser.TokenQueue.consume()
2,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,"org.jsoup.select.Evaluator$CssNthEvaluator.matches(org.jsoup.nodes.Element,org.jsoup.nodes.Element)"
3,SYMBOL_KIND_FIELD,SYMBOL_USE_INSTANCE_FIELD_READ,org.jsoup.nodes.Entities$EscapeMode.xhtml
6,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,org.jsoup.Connection$Request.timeout(int)
7,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,org.jsoup.nodes.Comment.nodeName()
8,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,org.jsoup.helper.ChangeNotifyingArrayList.removeAll(java.util.Collection)
10,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,org.jsoup.nodes.Node.traverse(org.jsoup.select.NodeVisitor)
11,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,org.jsoup.helper.HttpConnection.data(java.util.Collection)
14,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,org.jsoup.helper.HttpConnection$Request.postDataCharset()
16,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,org.jsoup.nodes.Document$OutputSettings.syntax()


### Unused API interactions in client

In [12]:
sum_not_seen = unique_uses(sum, suf_client)
print(len(sum_not_seen))
sum_not_seen.head(n=15)

2485


,usageKind,usageType,usageFullyQualifiedName
0,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,org.jsoup.select.Elements.addClass(java.lang.String)
1,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,org.jsoup.parser.TokenQueue.consume()
2,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,"org.jsoup.select.Evaluator$CssNthEvaluator.matches(org.jsoup.nodes.Element,org.jsoup.nodes.Element)"
3,SYMBOL_KIND_FIELD,SYMBOL_USE_INSTANCE_FIELD_READ,org.jsoup.nodes.Entities$EscapeMode.xhtml
4,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,org.jsoup.nodes.Node.hashCode()
6,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,org.jsoup.Connection$Request.timeout(int)
7,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,org.jsoup.nodes.Comment.nodeName()
8,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,org.jsoup.helper.ChangeNotifyingArrayList.removeAll(java.util.Collection)
10,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,org.jsoup.nodes.Node.traverse(org.jsoup.select.NodeVisitor)
11,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,org.jsoup.helper.HttpConnection.data(java.util.Collection)


### Most popular API symbols in all SUFs

In [13]:
suf.groupby(["usageFullyQualifiedName"]).size().sort_values(ascending=False).to_frame().head(n=15).style.bar()

,0
usageFullyQualifiedName,
org.jsoup.nodes.Element.select(java.lang.String),3172
org.jsoup.nodes.Document,1765
org.jsoup.nodes.Element,1733
org.jsoup.nodes.Element.text(),1582
org.jsoup.select.Elements.first(),1140
org.jsoup.select.Elements,1020
org.jsoup.nodes.Node.attr(java.lang.String),994
org.jsoup.Jsoup.parse(java.lang.String),851
org.jsoup.nodes.Element.html(),724


### Most popular API interactions

In [14]:
print("In all SUFs")
display(suf.groupby(["usageFullyQualifiedName","qualifiedUse"]).size().sort_values(ascending=False).to_frame().head(n=15).style.bar())
print("In clients")
display(suf_client.groupby(["usageFullyQualifiedName","qualifiedUse"]).size().sort_values(ascending=False).to_frame().head(n=15).style.bar())
print("In samples")
display(suf_sample.groupby(["usageFullyQualifiedName","qualifiedUse"]).size().sort_values(ascending=False).to_frame().head(n=15).style.bar())
print("In tests")
display(suf_test.groupby(["usageFullyQualifiedName","qualifiedUse"]).size().sort_values(ascending=False).to_frame().head(n=15).style.bar())

In all SUFs


In clients


In samples


In tests


### Least popular API symbols in all SUFs

  - Notice how some clients are invoking `*.internal.*` stuff!

In [15]:
suf.groupby(["usageFullyQualifiedName"]).size().sort_values(ascending=True).to_frame().head(n=15).style.bar()

,0
usageFullyQualifiedName,
org.jsoup.select.Selector$SelectorParseException(java.lang.String),1
org.jsoup.parser.CharacterReader(java.io.Reader),1
org.jsoup.Connection$KeyVal.inputStream(java.io.InputStream),1
"org.jsoup.internal.StringUtil.join(java.lang.String[],java.lang.String)",1
org.jsoup.Connection$KeyVal.value(java.lang.String),1
"org.jsoup.nodes.XmlDeclaration(java.lang.String,boolean)",1
org.jsoup.nodes.TextNode.createFromEncoded(java.lang.String),1
"org.jsoup.parser.CharacterReader(java.io.Reader,int)",1
org.jsoup.Connection$Request.proxy(java.net.Proxy),1


### Least popular API interactions in all SUFs

In [16]:
suf_client.groupby(["usageFullyQualifiedName","qualifiedUse"]).size().sort_values(ascending=True).to_frame().head(n=15).style.bar()

### Most popular interaction kinds in SUFs

In [17]:
print("In all SUFs")
display(suf["qualifiedUse"].value_counts().to_frame().style.bar())
print("In clients")
display(suf_client["qualifiedUse"].value_counts().to_frame().style.bar())
print("In samples")
display(suf_sample["qualifiedUse"].value_counts().to_frame().style.bar())
print("In tests")
display(suf_test["qualifiedUse"].value_counts().to_frame().style.bar())

In all SUFs


,qualifiedUse
SYMBOL_KIND_METHOD > SYMBOL_USE_VIRTUAL_INVOCATION,11609
SYMBOL_KIND_METHOD > SYMBOL_USE_INVOCATION,9740
SYMBOL_KIND_TYPE > SYMBOL_USE_TYPE_REFERENCE,5822
SYMBOL_KIND_METHOD > SYMBOL_USE_STATIC_INVOCATION,2109
SYMBOL_KIND_CONSTRUCTOR > SYMBOL_USE_INVOCATION,297
SYMBOL_KIND_CLASS > SYMBOL_USE_INSTANTIATION,297
SYMBOL_KIND_METHOD > SYMBOL_USE_OVERRIDING,75
SYMBOL_KIND_INTERFACE > SYMBOL_USE_IMPLEMENTATION,27
SYMBOL_KIND_CLASS > SYMBOL_USE_INHERITANCE,3


In clients


,qualifiedUse
SYMBOL_KIND_METHOD > SYMBOL_USE_VIRTUAL_INVOCATION,6157
SYMBOL_KIND_METHOD > SYMBOL_USE_INVOCATION,5101
SYMBOL_KIND_TYPE > SYMBOL_USE_TYPE_REFERENCE,3098
SYMBOL_KIND_METHOD > SYMBOL_USE_STATIC_INVOCATION,689
SYMBOL_KIND_CONSTRUCTOR > SYMBOL_USE_INVOCATION,76
SYMBOL_KIND_CLASS > SYMBOL_USE_INSTANTIATION,76
SYMBOL_KIND_METHOD > SYMBOL_USE_OVERRIDING,40
SYMBOL_KIND_INTERFACE > SYMBOL_USE_IMPLEMENTATION,7
SYMBOL_KIND_CLASS > SYMBOL_USE_INHERITANCE,2


In samples


,qualifiedUse
SYMBOL_KIND_METHOD > SYMBOL_USE_VIRTUAL_INVOCATION,84
SYMBOL_KIND_METHOD > SYMBOL_USE_INVOCATION,64
SYMBOL_KIND_TYPE > SYMBOL_USE_TYPE_REFERENCE,46
SYMBOL_KIND_METHOD > SYMBOL_USE_STATIC_INVOCATION,25
SYMBOL_KIND_METHOD > SYMBOL_USE_OVERRIDING,2
SYMBOL_KIND_INTERFACE > SYMBOL_USE_IMPLEMENTATION,1


In tests


,qualifiedUse
SYMBOL_KIND_METHOD > SYMBOL_USE_VIRTUAL_INVOCATION,5368
SYMBOL_KIND_METHOD > SYMBOL_USE_INVOCATION,4575
SYMBOL_KIND_TYPE > SYMBOL_USE_TYPE_REFERENCE,2678
SYMBOL_KIND_METHOD > SYMBOL_USE_STATIC_INVOCATION,1395
SYMBOL_KIND_CONSTRUCTOR > SYMBOL_USE_INVOCATION,221
SYMBOL_KIND_CLASS > SYMBOL_USE_INSTANTIATION,221
SYMBOL_KIND_METHOD > SYMBOL_USE_OVERRIDING,33
SYMBOL_KIND_INTERFACE > SYMBOL_USE_IMPLEMENTATION,19
SYMBOL_KIND_CLASS > SYMBOL_USE_INHERITANCE,1


### Debug: is there any interaction found in the SUF that do not exist in the SUM?

In [18]:
suf_not_seen = unique_uses(suf, sum)
print(len(suf_not_seen))
display(suf_not_seen)

0


,usageKind,usageType,usageFullyQualifiedName


## SUF comparisons

    - Funny thing: there are samples using `*.internal.*` types!

In [19]:
only_client_not_in_test = unique_uses(suf_client, suf_test)
print(f'{len(only_client_not_in_test)} interactions done in clients, not in tests')
display(only_client_not_in_test.head(15))

only_client_not_in_sample = unique_uses(suf_client, suf_sample)
print(f'{len(only_client_not_in_sample)} interactions done in clients, not in samples')
display(only_client_not_in_sample.head(15))

only_sample_not_in_client = unique_uses(suf_sample, suf_client)
print(f'{len(only_sample_not_in_client)} interactions done in samples, not in clients')
display(only_sample_not_in_client.head(15))

only_sample_not_in_test = unique_uses(suf_sample, suf_test)
print(f'{len(only_sample_not_in_test)} interactions done in samples, not in tests')
display(only_sample_not_in_test.head(15))

# Union of interactions that we find in every client 'kind'?

61 interactions done in clients, not in tests


,usageKind,usageType,usageFullyQualifiedName
36,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,org.jsoup.nodes.Node.equals(java.lang.Object)
80,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,org.jsoup.Connection.data(java.util.Map)
86,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,org.jsoup.Connection$Base.headers()
122,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,org.jsoup.nodes.Element.getElementsMatchingOwnText(java.util.regex.Pattern)
138,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,"org.jsoup.Connection$Base.header(java.lang.String,java.lang.String)"
139,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,"org.jsoup.Connection$Base.addHeader(java.lang.String,java.lang.String)"
140,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,org.jsoup.Connection$Base.hasHeader(java.lang.String)
141,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,"org.jsoup.Connection$Base.hasHeaderWithValue(java.lang.String,java.lang.String)"
142,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,org.jsoup.Connection$Base.removeHeader(java.lang.String)
143,SYMBOL_KIND_METHOD,SYMBOL_USE_OVERRIDING,org.jsoup.Connection$Base.headers()


397 interactions done in clients, not in samples


,usageKind,usageType,usageFullyQualifiedName
0,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,org.jsoup.nodes.Element.appendText(java.lang.String)
6,SYMBOL_KIND_TYPE,SYMBOL_USE_TYPE_REFERENCE,org.jsoup.Connection
10,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,org.jsoup.nodes.Element.selectFirst(java.lang.String)
12,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,org.jsoup.select.Elements.select(java.lang.String)
14,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,org.jsoup.nodes.Element.getElementsByClass(java.lang.String)
16,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,org.jsoup.select.Elements.attr(java.lang.String)
19,SYMBOL_KIND_TYPE,SYMBOL_USE_TYPE_REFERENCE,org.jsoup.nodes.DataNode
20,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,org.jsoup.nodes.Element.dataNodes()
21,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,org.jsoup.nodes.DataNode.getWholeData()
22,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,org.jsoup.Connection.data(java.lang.String[])


10 interactions done in samples, not in clients


,usageKind,usageType,usageFullyQualifiedName
2,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,"org.jsoup.Jsoup.parse(java.io.File,java.lang.String,java.lang.String)"
40,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,"org.jsoup.select.Elements.attr(java.lang.String,java.lang.String)"
41,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,org.jsoup.select.Elements.addClass(java.lang.String)
43,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,org.jsoup.nodes.Element.prepend(java.lang.String)
45,SYMBOL_KIND_METHOD,SYMBOL_USE_INVOCATION,org.jsoup.nodes.Element.wrap(java.lang.String)
46,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,org.jsoup.nodes.Node.wrap(java.lang.String)
62,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,"org.jsoup.select.Elements.attr(java.lang.String,java.lang.String)"
63,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,org.jsoup.select.Elements.addClass(java.lang.String)
65,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,org.jsoup.nodes.Element.prepend(java.lang.String)
67,SYMBOL_KIND_METHOD,SYMBOL_USE_VIRTUAL_INVOCATION,org.jsoup.nodes.Element.wrap(java.lang.String)


2 interactions done in samples, not in tests


,usageKind,usageType,usageFullyQualifiedName
18,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,"org.jsoup.internal.StringUtil.in(java.lang.String,java.lang.String[])"
19,SYMBOL_KIND_METHOD,SYMBOL_USE_STATIC_INVOCATION,"org.jsoup.helper.Validate.isTrue(boolean,java.lang.String)"
